In [5]:
import numpy as np
import pandas as pd
import json
import re
from collections import Counter
import little_mallet_wrapper
import seaborn
import glob
from pathlib import Path
#LOADING EVERYTHING
directory = "/Users/joshs/Desktop/Spring 2021/HUM346/wsj-nyt-oped/NYTimes"

In [1]:
#Importing stuff from NYTIMES
from pynytimes import NYTAPI
import urllib3, json, datetime
nyt = NYTAPI("nKBpzKBvOYipTxZf1T1NwCsVeXad5xoE", parse_dates=True)
articles = nyt.article_search(
    results = 500,
    dates = {
        "begin": datetime.datetime(2020, 10, 1),
        "end": datetime.datetime(2020, 10, 30)
    },
    options = {
        "sources": [
            "New York Times",
        ],
        "news_desk": [
            "OpEd",
            "Editorial"
            
        ]
    }
)

/Users/joshs/opt/anaconda3/lib/python3.8/site-packages/pynytimes/api.py:630: UserWarning: Asking for a lot of results, because of rate limits it can take a while.
  warnings.warn(


In [2]:
#Writing articles to be saved
for article in articles:
    
    text_file = open(f"{article['abstract']}.txt", "w+")
    n = text_file.write(article['lead_paragraph'] + " \n")
    text_file.close()


In [3]:
#splitting text into all lowercase
def split_into_words(any_chunk_of_text):
    lowercase_text = any_chunk_of_text.lower()
    split_words = re.split("\W+", lowercase_text) 
    return split_words

In [10]:
#Combining all data together IT SHOULD WORK NOW BUT WAS NOT WORKING EARLIER
files = glob.glob(f"{directory}/*.txt")
training_data = []
for file in files:
    text = open(file).read()
    processed_text = little_mallet_wrapper.process_string(text)
    training_data.append(processed_text)

In [4]:
original_texts = []
for file in files:
    text = open(file, encoding='utf-8').read()
    original_texts.append(text)

In [7]:

full_text = training_data
number_of_desired_words = 10
# Manipulate and Analyze File
stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours',
'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers',
 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are',
 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does',
 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into',
 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down',
 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here',
 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',
 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so',
 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', 've', 'll', 'amp']


all_the_words = split_into_words(full_text)
meaningful_words = [word for word in all_the_words if word not in stopwords]
meaningful_words_tally = Counter(meaningful_words)
most_frequent_meaningful_words = meaningful_words_tally.most_common(number_of_desired_words)

AttributeError: 'list' object has no attribute 'lower'

In [6]:
most_frequent_meaningful_words

[('many', 11),
 ('year', 9),
 ('new', 9),
 ('one', 9),
 ('world', 8),
 ('pandemic', 8),
 ('months', 8),
 ('coronavirus', 7),
 ('us', 7),
 ('states', 6)]

In [16]:

path_to_mallet = '/Users/joshs/mallet-2.0.8/bin/mallet'
num_topics = 10
    
#Change to your desired output directory
output_directory_path = '/Users/joshs/Desktop/Spring 2021/HUM346/wsj-nyt-oped/NYTimes/topic-model-output'

#No need to change anything below here
Path(f"{output_directory_path}").mkdir(parents=True, exist_ok=True)

path_to_training_data           = f"{output_directory_path}/training.txt"
path_to_formatted_training_data = f"{output_directory_path}/mallet.training"
path_to_model                   = f"{output_directory_path}/mallet.model.{str(num_topics)}"
path_to_topic_keys              = f"{output_directory_path}/mallet.topic_keys.{str(num_topics)}"
path_to_topic_distributions     = f"{output_directory_path}/{str(num_topics)}"


In [12]:
little_mallet_wrapper.print_dataset_stats(training_data)


Number of Documents: 328
Mean Number of Words per Document: 30.7
Vocabulary Size: 4306


In [17]:
#Importing Data
little_mallet_wrapper.import_data(path_to_mallet,
                path_to_training_data,
                path_to_formatted_training_data,
                training_data)

Importing data...
Complete


In [18]:
#Training Data Doesnt seem to work?
little_mallet_wrapper.train_topic_model(path_to_mallet,
                      path_to_formatted_training_data,
                      path_to_model,
                      path_to_topic_keys,
                      path_to_topic_distributions,
                      num_topics)

Training topic model...
Complete


In [19]:
topics = little_mallet_wrapper.load_topic_keys(path_to_topic_keys)

for topic_number, topic in enumerate(topics):
    print(f"✨Topic {topic_number}✨\n\n{topic}\n")

✨Topic 0✨

['two', 'could', 'one', 'might', 'national', 'decades', 'came', 'big', 'NUMs', 'security', 'argument', 'listen', 'talk', 'even', 'ever', 'idea', 'department', 'human', 'device', 'mobile']

✨Topic 1✨

['NUM', 'coronavirus', 'one', 'americans', 'first', 'pandemic', 'get', 'care', 'health', 'covid', 'time', 'year', 'days', 'early', 'nation', 'public', 'even', 'say', 'face', 'case']

✨Topic 2✨

['court', 'democrats', 'supreme', 'would', 'barrett', 'senate', 'coney', 'amy', 'political', 'power', 'republicans', 'president', 'judge', 'republican', 'conservative', 'system', 'win', 'confirmation', 'constitution', 'five']

✨Topic 3✨

['people', 'know', 'black', 'home', 'among', 'nearly', 'take', 'things', 'death', 'always', 'around', 'old', 'largest', 'used', 'others', 'back', 'point', 'half', 'mother', 'online']

✨Topic 4✨

['trump', 'president', 'donald', 'biden', 'joe', 'debate', 'presidential', 'last', 'week', 'election', 'question', 'back', 'never', 'going', 'thought', 'news', 'o